In [1]:
import numpy as np
from collections import defaultdict
import os

def initialize_lattice(L, p):
    return (np.random.rand(L, L) < p).astype(int)

def burning_method(lattice):
    L = lattice.shape[0]
    t = 2
    burning = np.zeros_like(lattice)
    burning[0, :] = (lattice[0, :] == 1) * t

    while t in burning:
        t += 1
        for i in range(L):
            for j in range(L):
                if burning[i, j] == t - 1:
                    for ni, nj in [(i-1, j), (i+1, j), (i, j-1), (i, j+1)]:
                        if 0 <= ni < L and 0 <= nj < L and lattice[ni, nj] == 1 and burning[ni, nj] == 0:
                            burning[ni, nj] = t
                            if ni == L - 1:
                                return True
    return False

def hoshen_kopelman(lattice):
    L = lattice.shape[0]
    label = np.zeros_like(lattice, dtype=int)
    cluster_sizes = defaultdict(int)
    current_label = 2

    for i in range(L):
        for j in range(L):
            if lattice[i, j] == 1:
                neighbors = []
                if i > 0 and label[i-1, j] > 1:
                    neighbors.append(label[i-1, j])
                if j > 0 and label[i, j-1] > 1:
                    neighbors.append(label[i, j-1])

                if not neighbors:
                    label[i, j] = current_label
                    cluster_sizes[current_label] += 1
                    current_label += 1
                else:
                    min_label = min(neighbors)
                    label[i, j] = min_label
                    cluster_sizes[min_label] += 1
                    for neighbor in neighbors:
                        if neighbor != min_label:
                            cluster_sizes[min_label] += cluster_sizes[neighbor]
                            cluster_sizes[neighbor] = -min_label

    final_sizes = defaultdict(int)
    for cluster, size in cluster_sizes.items():
        if size > 0:
            final_sizes[size] += 1

    return label, final_sizes

def monte_carlo_simulation(L, p, T):
    percolation_count = 0
    max_cluster_sizes = []
    cluster_distribution = defaultdict(int)

    for _ in range(T):
        lattice = initialize_lattice(L, p)
        percolates = burning_method(lattice)
        if percolates:
            percolation_count += 1

        _, cluster_sizes = hoshen_kopelman(lattice)
        max_cluster_sizes.append(max(cluster_sizes.keys()) if cluster_sizes else 0)

        for size, count in cluster_sizes.items():
            cluster_distribution[size] += count

    for size in cluster_distribution:
        cluster_distribution[size] /= T

    return percolation_count / T, np.mean(max_cluster_sizes), cluster_distribution

def load_input(filename="perc-ini.txt"):
    with open(filename, "r") as f:
        lines = f.readlines()
    params = {}
    for line in lines:
        parts = line.strip().split()
        if len(parts) >= 2:
            params[parts[1][1:]] = float(parts[0]) if '.' in parts[0] else int(parts[0])
    return params

def generate_output_files(params):
    L = params['L']
    T = params['T']
    p0 = params['p0']
    pk = params['pk']
    dp = params['dp']

    p_values = np.arange(p0, pk + dp, dp)
    results = []

    for p in p_values:
        pflow, avg_smax, cluster_dist = monte_carlo_simulation(L, p, T)
        results.append((p, pflow, avg_smax))

        # Write distribution file for this p
        dist_filename = f"Dist-p{p:.2f}-L{L}-T{T}.txt"
        with open(dist_filename, "w") as f:
            f.write("s  n(s, p, L)\n")
            for s, n_s in sorted(cluster_dist.items()):
                f.write(f"{s}  {n_s:.4f}\n")

    # Write average results file
    ave_filename = f"Ave-L{L}-T{T}.txt"
    with open(ave_filename, "w") as f:
        f.write("p  Pflow  ⟨smax⟩\n")
        for p, pflow, smax in results:
            f.write(f"{p:.4f}  {pflow:.4f}  {smax:.4f}\n")

def main():
    params = load_input()
    generate_output_files(params)

if __name__ == "__main__":
    main()


Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/opt/homebrew/anaconda3/lib/python3.11/site-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/var/folders/1r/p1_3kg652bbgwr9qm_hgg_pc0000gn/T/ipykernel_3475/3767675733.py", line 126, in <module>
    main()
  File "/var/folders/1r/p1_3kg652bbgwr9qm_hgg_pc0000gn/T/ipykernel_3475/3767675733.py", line 123, in main
    generate_output_files(params)
  File "/var/folders/1r/p1_3kg652bbgwr9qm_hgg_pc0000gn/T/ipykernel_3475/3767675733.py", line 104, in generate_output_files
    pflow, avg_smax, cluster_dist = monte_carlo_simulation(L, p, T)
                                    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/var/folders/1r/p1_3kg652bbgwr9qm_hgg_pc0000gn/T/ipykernel_3475/3767675733.py", line 68, in monte_carlo_simulation
    percolates = burning_method(lattice)
                 ^^^^^^^^^^^^^^^^^^^^^^^
  File "/var/folders/1r/p1_3kg652bbgwr9qm_hgg_pc0000gn/T/ipykernel_